In [1]:
import arcpy
import os
from dotenv import load_dotenv
from pathlib import Path

load_dotenv()
gdb_path = os.getenv('GDB_PATH')
# Convert the file path to a raw string
gdb_path = r"{}".format(gdb_path)
feature_class_path = os.path.join(gdb_path,"feature_class1")
feature_class_path = r"{}".format(feature_class_path)

In [2]:
### Check that both the geodatabase and feature class exist
if arcpy.Exists(gdb_path): 
    print(f"Geodatabase exists: {gdb_path}")
if arcpy.Exists(feature_class_path):
    print(f"Feature Class exists: {feature_class_path}" )

Geodatabase exists: C:\Users\jbfre\Documents\ArcGIS\Projects\From_Desert2_Oasis\From_Desert2_Oasis.gdb
Feature Class exists: C:\Users\jbfre\Documents\ArcGIS\Projects\From_Desert2_Oasis\From_Desert2_Oasis.gdb\feature_class1


In [21]:

### Get all the paths to the json coordinate data
grocery_data_path = Path(r"..\data_collection\grocery.json")
convenience_data_path = Path(r"..\data_collection\convenience.json")
cornerstore_data_path = Path(r"..\data_collection\cornerstore.json")
drugs_nliquor_data_path = Path(r"..\data_collection\drugs_nliquor.json")
farm_market_data_path = Path(r"..\data_collection\farm_market.json")
fast_food_data_path = Path(r"..\data_collection\fast_food.json")
food_bank_data_path = Path(r"..\data_collection\food_bank.json")
mini_mart_data_path = Path(r"..\data_collection\mini_mart.json")

grocery_data_path_2 =  Path(r"..\data_collection\grocery_edited.json")

DATA_PATH_LIST = [convenience_data_path,
                  cornerstore_data_path,
                  fast_food_data_path,
                  mini_mart_data_path
                  ]

for path in DATA_PATH_LIST:
    if path.exists():
        print(f"{path.name} exists")
    else:
        print(f"{path.name} doesn't exist")


convenience.json exists
cornerstore.json exists
fast_food.json exists
mini_mart.json exists


In [26]:
### Load in json data and plot on feature layer
import json
import arcpy
from arcpy.da import InsertCursor
from shapely.geometry import Point

spatial_ref = arcpy.SpatialReference(4326) # WGS 84 (latitude/longitude)
SOURCE_FEATURE_CLASS = feature_class_path
SOURCE_GDB = gdb_path

In [5]:
for field in arcpy.ListFields(feature_class_path):
    print(field.name)

OBJECTID
Shape
Color


In [22]:
# Define the output workspace (e.g., a geodatabase or folder)
output_workspace = SOURCE_GDB

# Define the name of the new feature class
feature_class_name = "healthy_alt"

# Define the geometry type (Point, Polyline, Polygon, etc.)
geometry_type = "POINT"

# Define the spatial reference (e.g., WGS 1984)
spatial_reference = arcpy.SpatialReference(4326)  # WGS 1984

# Create the feature class
arcpy.CreateFeatureclass_management(
    output_workspace,
    feature_class_name,
    geometry_type,
    spatial_reference=spatial_reference
)

ExecuteError: ERROR 000258: Output C:\Users\jbfre\Documents\ArcGIS\Projects\From_Desert2_Oasis\From_Desert2_Oasis.gdb\healthy_alt already exists
Failed to execute (CreateFeatureclass).


In [27]:
feature_class_path_2 = os.path.join(gdb_path,"healthy_alt")
feature_class_path_2 = r"{}".format(feature_class_path_2)



In [29]:
with open(farm_market_data_path,'r') as file:
    farm_market_json = json.load(file)

with open(food_bank_data_path,'r') as file:
    food_bank_json = json.load(file)


json_list = [farm_market_json,food_bank_json]
healthy_alt = ['Shape','Color']

# Add the 'Color' field to the feature class if it doesn't already exist
if 'Color' not in [f.name for f in arcpy.ListFields(feature_class_path_2)]:
    arcpy.AddField_management(feature_class_path_2, 'Color', 'TEXT', field_length=50)

for json in json_list:
    # Create an insert cursor to add data to the feature class
    with arcpy.da.InsertCursor(feature_class_path_2,healthy_alt) as cursor:
        for address, coordinates in json.items():
            lat = coordinates[0]
            lon = coordinates[1]

            # Create a point geometry (WGS84)
            point = arcpy.Point(lon, lat)
            point_geometry = arcpy.PointGeometry(point, spatial_ref)

            # Insert the point into the feature class
            cursor.insertRow([point_geometry,'green'])

##### Creating Feature Layers for all of the different food options

In [5]:
### Grocery Store Code

# Define paths

with open(grocery_data_path_2,'r') as file:
    grocery_json = json.load(file)

grocery_buffer_layer = os.path.join(SOURCE_GDB,"grocery_buffer_layer_2v1")

# Create a feature layer from the existing feature class
arcpy.MakeFeatureLayer_management(feature_class_path_2, "grocery_lyr_2v1")
print("Feature layer created.")

Feature layer created.


In [6]:
# Insert points into the feature class
grocery_fields = ['Shape', 'Color']  # Fields for shape and address
with arcpy.da.InsertCursor(feature_class_path_2, grocery_fields) as cursor:
    for address, coordinates in grocery_json.items():
        lat, lon = coordinates  # Latitude and Longitude
        point = arcpy.Point(lon, lat)  # Create point
        point_geometry = arcpy.PointGeometry(point, spatial_ref)  # Convert to point geometry
        cursor.insertRow([point_geometry,'green'])  # Insert the row

In [7]:
# Create 2-mile buffers around the points
arcpy.analysis.Buffer(feature_class_path_2, grocery_buffer_layer, "0.5 miles")

print("Buffers created around points.")


Buffers created around points.


In [8]:
# Create a feature layer for the buffer
arcpy.MakeFeatureLayer_management(grocery_buffer_layer, "grocery_buffer_lyr_2v1")
print("Buffer layer created.")


Buffer layer created.


In [ ]:

# # Optional: If you want to view both the points and buffers together
# arcpy.management.AddLayer("points_lyr", "TOP")  # Adds points to the map
# arcpy.management.AddLayer("buffer_lyr", "TOP")  # Adds buffer to the map
# print("Both points and buffers added to the map.")